In [21]:
import pandas as pd

In [22]:
df = pd.read_csv("src/data/normalization_assesment_dataset_10k.csv")
print(df)

                                  raw_comp_writers_text  \
0               Jordan Riley/Adam Argyle/Martin Brammer   
1                                         Martin Hygård   
2     Jesse Robinson/Greg Phillips/Kishaun Bailey/Ka...   
3                                        Mendel Brikman   
4                                             Alvin Lee   
...                                                 ...   
9995                                      Terrance Byrd   
9996                                       Pillar Point   
9997  FRANK BRIAN KEITH (MR)/JENNINGS JOHN JOSEPH IV...   
9998                                       Sierra Lucas   
9999                         Budde Music/Lorenz Brunner   

                                             CLEAN_TEXT  
0               Jordan Riley/Adam Argyle/Martin Brammer  
1                                         Martin Hygård  
2     Jesse Robinson/Greg Phillips/Kishaun Bailey/Ka...  
3                                                   NaN  
4

**Redundant entries:** Many CLEAN_TEXT values are identical to raw_comp_writers_text

In [23]:

print("CLEAN_TEXT values identical to raw_comp_writers_text:",df[df['raw_comp_writers_text'] == df['CLEAN_TEXT']].shape[0])
print("percentage of CLEAN_TEXT values identical to raw_comp_writers_text:",(df[df['raw_comp_writers_text'] == df['CLEAN_TEXT']].shape[0]/df.shape[0]) * 100,"%")

print(df[df['raw_comp_writers_text'] == df['CLEAN_TEXT']])

CLEAN_TEXT values identical to raw_comp_writers_text: 6323
percentage of CLEAN_TEXT values identical to raw_comp_writers_text: 63.23 %
                                  raw_comp_writers_text  \
0               Jordan Riley/Adam Argyle/Martin Brammer   
1                                         Martin Hygård   
2     Jesse Robinson/Greg Phillips/Kishaun Bailey/Ka...   
4                                             Alvin Lee   
6                                          Mefi Morales   
...                                                 ...   
9992  Ludovic Berrou/Jean-Christophe Colliou/Damien ...   
9993  Jonny Buckland/Guy Berryman/Will Champion/Chri...   
9995                                      Terrance Byrd   
9996                                       Pillar Point   
9998                                       Sierra Lucas   

                                             CLEAN_TEXT  
0               Jordan Riley/Adam Argyle/Martin Brammer  
1                                       

**Missing values:** Some rows have NaN values.

In [24]:
print("Empty CLEAN_TEXT values:",df[df['CLEAN_TEXT'].isnull()].shape[0])
print("percentage of Empty CLEAN_TEXT values:",(df[df['CLEAN_TEXT'].isnull()].shape[0]/df.shape[0]) * 100,"%")

print(df[df['CLEAN_TEXT'].isnull()])


Empty CLEAN_TEXT values: 1341
percentage of Empty CLEAN_TEXT values: 13.41 %
                                  raw_comp_writers_text CLEAN_TEXT
3                                        Mendel Brikman        NaN
8                               UNKNOWN WRITER (999990)        NaN
25                                               fo man        NaN
27                                          Traditional        NaN
28                                            #unknown#        NaN
...                                                 ...        ...
9968                                   Phanendra/Shakti        NaN
9974  AEPI/Hamburge, Darjean/Hamburge, Darjean, Sevi...        NaN
9976  Orhan Gencebay, Yaacov Lamai, Dolev Ram, Penn ...        NaN
9979                             Galkin A./Afanas'ev V.        NaN
9981                                              VCPMC        NaN

[1341 rows x 2 columns]


In [25]:
print("Empty raw_comp_writers_text values:",df[df['raw_comp_writers_text'].isnull()].shape[0])
print("percentage of Empty raw_comp_writers_text values:",(df[df['raw_comp_writers_text'].isnull()].shape[0]/df.shape[0]) * 100,"%")

print(df[df['raw_comp_writers_text'].isnull()])

Empty raw_comp_writers_text values: 1
percentage of Empty raw_comp_writers_text values: 0.01 %
    raw_comp_writers_text CLEAN_TEXT
165                   NaN        NaN


In [26]:
from collections import Counter
import re
#split raw_comp_writers_text into names delimited by /
names_raw = []
for text in df['raw_comp_writers_text']:
    if type(text) == float:
        continue
    names_raw += re.split(r'/', text)

name_freq_raw = Counter(names_raw)
print(name_freq_raw.most_common(50))
#len of names_freq_raw
print(len(name_freq_raw))

[('<Unknown>', 341), ('Copyright Control', 119), ('Traditional', 21), ('Wolfgang Amadeus Mozart', 20), ('COPYRIGHT CONTROL', 20), ('Sameer', 16), ('Giuseppe Verdi', 14), ('Johann Sebastian Bach', 11), ('[traditional]', 10), ('Ludwig van Beethoven', 9), ('Billie Ray Fingers', 9), ('Bruce Fingers', 9), ('Thomas Bergersen', 9), ('ZUN', 9), ('John Lennon', 8), ('Paul McCartney', 8), ('Juice WRLD', 8), ('Anand Bakshi', 8), ('LEEZ', 8), ('Ollounder', 8), ('George Gershwin', 8), ('Savan Kotecha', 7), ('WRITER UNKNOWN', 7), ('Robert Hunter', 7), ('Not Documented', 7), ('Richard Strauss', 7), ('A. R. Rahman', 7), ('김홍중', 7), ('EDEN', 7), ('송민기', 7), ('Max Martin', 7), ('Nadeem-Shravan', 6), ('Jesús María Corman', 6), ('Mikel Erentxun', 6), ('Andrew Lloyd Webber', 6), ('Antonio Vivaldi', 6), ('Phạm Duy', 6), ('Ritchie Blackmore', 6), ('Phil Collins', 6), ('Lorenzo Da Ponte', 6), ('Franz Schubert', 6), ('Lars Ulrich', 6), ('James Hetfield', 6), ('Sony', 6), ('Sonoton Music GmbH & Co. KG', 6), ('T

In [ ]:
import re
import pandas as pd

# Define noisy names to be removed
NOISY_NAMES = [
    r'<Unknown>', r'Copyright Control', r'Traditional', r'COPYRIGHT CONTROL', 
    r'\[traditional\]', r'WRITER UNKNOWN', r'Not Documented', 
    r'Sonoton Music GmbH & Co. KG', r'UNKNOWN WRITER \(999990\)', 
    r'UNKNOWN WRITER', r'PUBLISHER UNKNOWN', 
    r'Figurata Music GmbH & Co. KG', r'#unknown#'
]

def clean_noisy_names(text):
    if not isinstance(text, str):
        return text
    for noisy in NOISY_NAMES:
        text = re.sub(noisy, '', text, flags=re.IGNORECASE)  
    return text.strip()  

def preprocess_text_column(df, column_name):
    """
    Preprocesses a text column in a DataFrame by removing noisy names, cleaning delimiters,
    and removing empty rows.
    
    Args:
        df (pd.DataFrame): The DataFrame containing the text column.
        column_name (str): The name of the column to preprocess.
        
    Returns:
        pd.DataFrame: The cleaned DataFrame.
    """
    # Apply cleaning function to the column
    df[column_name] = df[column_name].apply(clean_noisy_names)

    # Remove extra delimiters left after cleaning
    df[column_name] = df[column_name].str.replace(r'[/,]\s*$', '', regex=True)

    # Remove double delimiters //
    df[column_name] = df[column_name].str.replace(r'//', '/', regex=True)

    # Remove leading and trailing /
    df[column_name] = df[column_name].str.replace(r'^/|/$', '', regex=True)

    # Remove empty rows and rows with empty strings
    df = df.dropna(subset=[column_name])
    df = df[df[column_name] != '']

    # Reset index for cleanliness
    df = df.reset_index(drop=True)

    return df

# Example usage
df = pd.read_csv("src/data/normalization_assesment_dataset_10k.csv")  # Load your dataset
df = preprocess_text_column(df, 'raw_comp_writers_text')  # Preprocess the column

df_test = df.sample(frac=0.99, random_state=200)
df_examples = df.drop(df_test.index)
# Save the cleaned DataFrame to a CSV file
df_examples.to_csv("src/data/normalization_assesment_dataset_10k_cleaned_examples.csv", index=False)
df_test["raw_comp_writers_text"].to_csv("src/data/normalization_assesment_dataset_10k_cleaned_test.csv", index=False)
df.to_csv("src/data/normalization_assesment_dataset_10k_cleaned.csv", index=False)